In [1]:
import pandas as pd
import json
import numpy as np
# prod_arg = ['category', 'description', 'title', 'brand', 'main_cat', 'price', 'asin', 'imageURL', 'imageURLHighRes']
# rat_arg=['overall', 'reviewerID','asin', 'reviewerName']
rating_df = pd.read_json('Tools_and_Home_Improvement.json', lines=True, nrows=150000)
# rating_df.rename(columns={"0006564224":"asin","A3NSN9WOX8470M":"reviewerID","5.0":"overall"}, inplace=True)
# rating_df = rating_df[['overall', 'reviewerID','asin', 'reviewerName']]
products_df = pd.read_json('meta_Tools_and_Home_Improvement.json', lines=True, nrows=150000)
products_df = products_df[['category', 'description', 'title', 'brand', 'main_cat', 'price', 'asin', 'imageURL', 'imageURLHighRes']]

In [2]:
print(rating_df.shape)
print(products_df.shape)
rating_df.tail(5)

(150000, 12)
(150000, 9)


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
149995,2,True,"05 26, 2016",ALZGYJEG9VRXJ,B00004Z4AV,{'Size Name:': ' 1'},Hockey Mom,Nice sanding sponge HOWEVER they disintegrated...,Not the best quality,1464220800,NaN,NaN
149996,4,True,"05 21, 2016",AXESSTKHDO3FE,B00004Z4AV,{'Size Name:': ' 3-Pack'},YanAmazon,Way better than straight sanding paper. Howev...,Way better than straight sanding paper,1463788800,NaN,NaN
149997,3,True,"05 12, 2016",A4Y4GZVAXF2CX,B00004Z4AV,{'Size Name:': ' 1'},NASOTANG,Good sanding block but it's cheaper at Home De...,Three Stars,1463011200,NaN,NaN
149998,5,True,"05 11, 2016",A2SQ7E7ZEZEEUZ,B00004Z4AV,{'Size Name:': ' 1'},EM,"These sanders are so good, I've already ordere...",Five Stars,1462924800,NaN,NaN
149999,5,True,"04 27, 2016",A3H6IZMBRR6CRY,B00004Z4AV,{'Size Name:': ' 1'},Classy Costie,Great device to help get old paint off the wal...,Great tool,1461715200,NaN,NaN


In [4]:
rating_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         150000 non-null  int64 
 1   verified        150000 non-null  bool  
 2   reviewTime      150000 non-null  object
 3   reviewerID      150000 non-null  object
 4   asin            150000 non-null  object
 5   style           85513 non-null   object
 6   reviewerName    149998 non-null  object
 7   reviewText      149955 non-null  object
 8   summary         149983 non-null  object
 9   unixReviewTime  150000 non-null  int64 
 10  vote            23019 non-null   object
 11  image           1341 non-null    object
dtypes: bool(1), int64(2), object(9)
memory usage: 12.7+ MB


# Popularity based recommendations

In [5]:
rat_count_df=rating_df.groupby('asin').count()['overall'].reset_index()
rat_count_df.rename(columns={"overall":"rating-count"}, inplace=True)
rat_count_df

,asin,rating-count
0,0972970169,5
1,0982085028,275
2,0997438207,22
3,1590651529,1
4,1590651669,20
...,...,...
1816,B00004Z2Y0,32
1817,B00004Z49T,4
1818,B00004Z4AV,16
1819,B00004Z4DU,83


In [6]:
rat_avg_df = rating_df.groupby('asin').mean()['overall'].reset_index()
rat_avg_df.rename(columns={"overall":"rating-avg"}, inplace=True)
rat_avg_df


C:\Users\surya\AppData\Local\Temp\ipykernel_14780\3651525842.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rat_avg_df = rating_df.groupby('asin').mean()['overall'].reset_index()


,asin,rating-avg
0,0972970169,4.000000
1,0982085028,4.690909
2,0997438207,4.863636
3,1590651529,5.000000
4,1590651669,3.050000
...,...,...
1816,B00004Z2Y0,4.468750
1817,B00004Z49T,4.500000
1818,B00004Z4AV,4.375000
1819,B00004Z4DU,4.457831


In [7]:
popular_df = rat_avg_df.merge(rat_count_df,on="asin")
popular_df = popular_df[popular_df['rating-count'] > 200].sort_values('rating-count', ascending=False).reset_index()
popular_df

,index,asin,rating-avg,rating-count
0,1724,B00004YVAJ,4.694554,5821
1,1461,B00004U9JP,4.252815,4707
2,11,193362275X,4.452081,3725
3,854,B00002NB2X,4.037143,3500
4,664,B00002N6JC,4.499785,2327
...,...,...,...,...
148,904,B00002X1XD,4.425743,202
149,901,B00002X1X4,4.425743,202
150,1068,B00004R9LH,4.485149,202
151,305,B0000223XZ,3.641791,201


In [8]:
best_products = products_df.merge(popular_df,on="asin").sort_values('rating-count', ascending=False).reset_index()
# display(best_products.head(10))
print(best_products.shape)
best_products['title'].drop_duplicates(inplace=True)
best_products

(299, 13)


,level_0,category,description,title,brand,main_cat,price,asin,imageURL,imageURLHighRes,index,rating-avg,rating-count
0,293,"[Tools & Home Improvement, Power & Hand Tools,...","[The 2 1/4"" Classic SD is the epitome of Swiss...","Victorinox Swiss Army Classic SD Pocket Knife,...",Victorinox,Sports & Outdoors,$17.61,B00004YVAJ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1724,4.694554,5821
1,294,"[Tools & Home Improvement, Power & Hand Tools,...","[The 2 1/4"" Classic SD is the epitome of Swiss...","Victorinox Swiss Army Classic SD Pocket Knife,...",Victorinox,Sports & Outdoors,$17.61,B00004YVAJ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1724,4.694554,5821
2,251,"[Tools & Home Improvement, Rough Plumbing, Gar...",[For over 75 years InSinkErator has been known...,Insinkerator Badger 1 Garbage Disposal 1/3HP,InSinkErator,Industrial & Scientific,$5.71,B00004U9JP,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1461,4.252815,4707
3,252,"[Tools & Home Improvement, Rough Plumbing, Gar...",[For over 75 years InSinkErator has been known...,Insinkerator Badger 1 Garbage Disposal 1/3HP,InSinkErator,Industrial & Scientific,$5.71,B00004U9JP,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1461,4.252815,4707
4,1,"[Tools & Home Improvement, Lighting & Ceiling ...",[Who said you cant craft while enjoying family...,"Mighty Bright 40516 XtraFlex2 Book Light, Pink",Mighty Bright,Office Products,$27.00,193362275X,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,11,4.452081,3725
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,183,"[Tools & Home Improvement, Power & Hand Tools,...","[Bosch TS1007 Dado Insert for 10"" Table SawBos...",Bosch TS1007 Dado Insert,Bosch,Tools & Home Improvement,$10.00,B00004R9LH,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1068,4.485149,202
295,36,"[Tools & Home Improvement, Power & Hand Tools,...",[Woodstock Mini 2-Stage Cyclone Separator W204...,Woodstock W2049 Mini 2-Stage Cyclone Separator,Woodstock,Tools & Home Improvement,$18.02,B0000223XZ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,305,3.641791,201
296,286,"[Tools & Home Improvement, Electrical, Switche...","[WIRELESS 3-WAY DECORA, The Leviton Anywhere S...",Leviton Anywhere Decora RF Remote 3-way switch...,Leviton,Tools & Home Improvement,,B00004YUOL,[],[],1719,3.213930,201
297,285,"[Tools & Home Improvement, Electrical, Switche...","[WIRELESS 3-WAY DECORA, The Leviton Anywhere S...",Leviton Anywhere Decora RF Remote 3-way switch...,Leviton,Tools & Home Improvement,,B00004YUOL,[],[],1719,3.213930,201


# ITEM - ITEM collaborative filtering

In [20]:
temp_y = rating_df.groupby('reviewerID').count()['overall'] > 2
customerbyvote = temp_y[temp_y].index
customerbyvote

Index(['A10063PJ5C9WQQ', 'A100MU000SQ0QB', 'A100WO06OQR8BQ', 'A103Q714AOKOA4',
       'A107LJVG2BC22R', 'A10BC5NHJXLZL0', 'A10BGHYI4KOZVF', 'A10D12KO4PY8DB',
       'A10FFY8Z9M2NSQ', 'A10K9EBCX8V2NN',
       ...
       'AZI2J4O6TDKYP', 'AZJ5K7J8C9VOP', 'AZP8DAHJCFWPH', 'AZQ5JI9UYMQ70',
       'AZSR0OON5RAGK', 'AZSUZOUC080Q0', 'AZTR7IC4Z9PX5', 'AZUZROST1AGB7',
       'AZVFHFGNPPSCC', 'AZW2RVW8FK2C6'],
      dtype='object', name='reviewerID', length=2866)

In [21]:
filtered_user = rating_df[rating_df['reviewerID'].isin(customerbyvote)]
filtered_user

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
59,4,True,"12 12, 2016",A32UGULVMQUN33,0982085028,{'Style:': ' 3) IR260 POU (30A/6kW/220v)'},J. K. Dresser,Work well. The only complaint I have is that ...,Works well with one exception,1481500800,11,NaN
406,5,False,"12 22, 2007",A39SJ6V6QCGC37,193362275X,NaN,Midge,I bought three of these in blue (a spare for m...,Excellent product!!!!!!!!!!,1198281600,223,NaN
407,5,False,"12 22, 2007",A39SJ6V6QCGC37,193362275X,NaN,Midge,I bought three of these in blue (a spare for m...,Excellent product!!!!!!!!!,1198281600,23,NaN
455,5,True,"11 28, 2017",A25L5YZNUZHON6,193362275X,{'Format:': ' Kitchen'},From NJ,"Not my first book light, sad to admit, but thi...","Mighty Bright, Mighty Purple, Mighty Happy!",1511827200,NaN,NaN
603,4,True,"12 3, 2016",A3H9DSJS96YY84,193362275X,{'Format:': ' Electronics'},Mayela,"Yes, good product for the readers.",good product for the readers,1480723200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
149938,5,True,"12 12, 2017",A33S1KUA5WNAVC,B00004Z4DU,"{'Size Name:': ' 1.88-Inch x 60-Yard', 'Style:...",tgiv,Worked great on F150 grill repainting,Only the best,1513036800,NaN,NaN
149939,5,True,"12 7, 2017",A2A444MFNFFYAV,B00004Z4DU,"{'Size Name:': ' .94-Inch x 60 -Yard', 'Style:...",Question,"It's painter's tape, all right.",Five Stars,1512604800,NaN,NaN
149950,4,True,"06 17, 2013",A1QLQCJVGCQPP0,B00004Z4AV,{'Size Name:': ' 1'},Brock Rumer,These sanding sponges are really useful to hav...,Makes the job faster,1371427200,2,NaN
149963,4,True,"04 9, 2016",A3T4TN2OSDOKJ5,B00004Z4H5,NaN,K. Braswell,"Drill attachment paint mixer. It works, but bl...",Does adequate job,1460160000,NaN,NaN


In [28]:
temp_x = filtered_user.groupby('asin').count()['overall'] > 5
products_gt_50 = temp_x[temp_x].index
products_gt_50

Index(['1933622741', '193362275X', '7109036146', '7300025137', '7502175741',
       'B0000222UX', 'B0000222V1', 'B0000222V3', 'B0000222V6', 'B0000222V7',
       ...
       'B00004YVAE', 'B00004YVAJ', 'B00004YVB4', 'B00004YYCW', 'B00004YZNM',
       'B00004Z0YR', 'B00004Z0ZJ', 'B00004Z13K', 'B00004Z2WZ', 'B00004Z4DU'],
      dtype='object', name='asin', length=464)

In [29]:
filtered_product = filtered_user[filtered_user['asin'].isin(products_gt_50)]
filtered_product = filtered_product.merge(products_df, on='asin')[['asin', 'overall', 'reviewerID', 'title', 'price', 'category', 'imageURLHighRes']]
filtered_product

,asin,overall,reviewerID,title,price,category,imageURLHighRes
0,193362275X,5,A39SJ6V6QCGC37,"Mighty Bright 40516 XtraFlex2 Book Light, Pink",$27.00,"[Tools & Home Improvement, Lighting & Ceiling ...",[https://images-na.ssl-images-amazon.com/image...
1,193362275X,5,A39SJ6V6QCGC37,"Mighty Bright 40516 XtraFlex2 Book Light, Pink",$27.00,"[Tools & Home Improvement, Lighting & Ceiling ...",[https://images-na.ssl-images-amazon.com/image...
2,193362275X,5,A25L5YZNUZHON6,"Mighty Bright 40516 XtraFlex2 Book Light, Pink",$27.00,"[Tools & Home Improvement, Lighting & Ceiling ...",[https://images-na.ssl-images-amazon.com/image...
3,193362275X,4,A3H9DSJS96YY84,"Mighty Bright 40516 XtraFlex2 Book Light, Pink",$27.00,"[Tools & Home Improvement, Lighting & Ceiling ...",[https://images-na.ssl-images-amazon.com/image...
4,193362275X,4,A1YEB5ILHJW0NT,"Mighty Bright 40516 XtraFlex2 Book Light, Pink",$27.00,"[Tools & Home Improvement, Lighting & Ceiling ...",[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...
18214,B00004Z4DU,2,A2R79MGPOIP6B9,"ScotchBlue Painter's Tape, Multi-Use, 1.88-Inc...",$6.29,"[Tools & Home Improvement, Paint, Wall Treatme...",[https://images-na.ssl-images-amazon.com/image...
18215,B00004Z4DU,5,A33S1KUA5WNAVC,"ScotchBlue Painter's Tape, Multi-Use, 1.88-Inc...",$6.29,"[Tools & Home Improvement, Paint, Wall Treatme...",[https://images-na.ssl-images-amazon.com/image...
18216,B00004Z4DU,5,A33S1KUA5WNAVC,"ScotchBlue Painter's Tape, Multi-Use, 1.88-Inc...",$6.29,"[Tools & Home Improvement, Paint, Wall Treatme...",[https://images-na.ssl-images-amazon.com/image...
18217,B00004Z4DU,5,A2A444MFNFFYAV,"ScotchBlue Painter's Tape, Multi-Use, 1.88-Inc...",$6.29,"[Tools & Home Improvement, Paint, Wall Treatme...",[https://images-na.ssl-images-amazon.com/image...


In [30]:
pivottable = filtered_product.pivot_table(index='asin', columns='reviewerID', values='overall')
pivottable.fillna(0,inplace=True)
pivottable

reviewerID,A10063PJ5C9WQQ,A100MU000SQ0QB,A100WO06OQR8BQ,A103Q714AOKOA4,A107LJVG2BC22R,A10BC5NHJXLZL0,A10BGHYI4KOZVF,A10D12KO4PY8DB,A10FFY8Z9M2NSQ,A10K9EBCX8V2NN,...,AZI2J4O6TDKYP,AZJ5K7J8C9VOP,AZP8DAHJCFWPH,AZQ5JI9UYMQ70,AZSR0OON5RAGK,AZSUZOUC080Q0,AZTR7IC4Z9PX5,AZUZROST1AGB7,AZVFHFGNPPSCC,AZW2RVW8FK2C6
asin,,,,,,,,,,,,,,,,,,,,,
1933622741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193362275X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7109036146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7300025137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7502175741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B00004Z0YR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004Z0ZJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004Z13K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_dist=cosine_similarity(pivottable)
cosine_dist

array([[1.       , 0.4526169, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.4526169, 1.       , 0.0525079, ..., 0.042232 , 0.       ,
        0.       ],
       [0.       , 0.0525079, 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.042232 , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [35]:
def recommender(item_name):
    index = np.where(pivottable.index==item_name)[0][0]
    similar = list(enumerate(cosine_dist[index]))
    similar.sort(key=lambda a:a[1], reverse=True)
    data=[]
    for i in similar[1:6]:
#         item=[]
#         temp_df=popular_df[best_products['asin']==pivottable.index[i[0]]]
#         item.extend(list(temp_df.drop_duplicates('title')['asin'].values))
#         item.extend(list(temp_df.drop_duplicates('title')['rating-avg'].values))
#         item.extend(list(temp_df.drop_duplicates('title')['rating-count'].values))
#         data.append(item)
        k=pivottable.index[i[0]]
        print('*'*100)
        print(k)
        print(best_products.loc[best_products['asin'] == k]['title'].to_string())

#     return data

In [36]:
item = "193362275X"
print(best_products.loc[best_products['asin'] == item]['title'].to_string())
print("=============================")
recommender(item)

4    Mighty Bright 40516 XtraFlex2 Book Light, Pink
****************************************************************************************************
1933622741
278    Mighty Bright 40514 XtraFlex2 Book Light, Green
****************************************************************************************************
B00002N6MY
172    Korky 528 Toilet Fill Valve, Universal, Black
173    Korky 528 Toilet Fill Valve, Universal, Black
****************************************************************************************************
B00004TR4Q
46    Gerber MP600 Multi-Plier, Blunt Nose, Stainles...
47    Gerber MP600 Multi-Plier, Blunt Nose, Stainles...
****************************************************************************************************
B0000224M4
Series([], )
****************************************************************************************************
B00002X1ZJ
Series([], )


In [37]:
popular_df

,index,asin,rating-avg,rating-count
0,1724,B00004YVAJ,4.694554,5821
1,1461,B00004U9JP,4.252815,4707
2,11,193362275X,4.452081,3725
3,854,B00002NB2X,4.037143,3500
4,664,B00002N6JC,4.499785,2327
...,...,...,...,...
148,904,B00002X1XD,4.425743,202
149,901,B00002X1X4,4.425743,202
150,1068,B00004R9LH,4.485149,202
151,305,B0000223XZ,3.641791,201


In [38]:
import pickle
pickle.dump(popular_df, open('popular_df_ht.pkl','wb'))

In [39]:
import pickle
pickle.dump(best_products, open('best_products_ht.pkl','wb'))

In [40]:
import pickle
pickle.dump(cosine_dist, open('cosine_dist_ht.pkl','wb'))

In [41]:
import pickle
pickle.dump(pivottable, open('pivottable_ht.pkl','wb'))